## 计算向量，统计结果等

## 8. 获得基于第二阶段模型所得标问向量

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append('/tf/libo/Project/im/exe')
from task_sentence_embedding_FinanceFAQ_step2_1 import model
from config import *

#get list
std_data=pd.read_csv(std_data_path, sep="\t")
q_std_list=std_data.drop_duplicates("q_std").q_std.tolist()
q_sim_list=std_data.drop_duplicates("q_sim").q_sim.tolist()
q_corpus=pd.DataFrame(q_std_list+q_sim_list,columns=['q']).drop_duplicates("q").q.tolist()
q_std_df=pd.DataFrame(q_std_list,columns=['q'])
q_corpus_df=pd.DataFrame(q_corpus,columns=['q'])
q_sim_df=pd.DataFrame(q_sim_list,columns=['q'])
print('保存标问：', q_std_df.shape, os.path.join(data_dir,sec_q_std_file))
q_std_df.to_csv(os.path.join(data_dir,sec_q_std_file),index=None,header=False,sep="\t")
print('保存所有语料：', q_corpus_df.shape, os.path.join(data_dir,sec_q_corpus_file))
q_corpus_df.to_csv(os.path.join(data_dir,sec_q_corpus_file),index=None,header=False,sep="\t")
print('保存相似问：', q_sim_df.shape, os.path.join(data_dir,sec_q_sim_file))
q_sim_df.to_csv(os.path.join(data_dir,sec_q_sim_file),index=None,header=False,sep="\t")
print('q_std_list:——>',len(q_std_list),'q_sim_list:——>',len(q_sim_list),'q_corpus:——>',len(q_corpus))
#get embeddings
q_std_sentence_embeddings = model.encode(q_std_list,batch_size=64)
print('保存标准问向量：', os.path.join(data_dir,sec_q_std_vectors_file))
np.save(os.path.join(data_dir,sec_q_std_vectors_file),q_std_sentence_embeddings)
q_corpus_sentence_embeddings = model.encode(q_corpus,batch_size=64)
print('保存所有语料向量：', os.path.join(data_dir,sec_q_corpus_vectors_file))
np.save(os.path.join(data_dir,sec_q_corpus_vectors_file),q_corpus_sentence_embeddings)


9. 获得所有待测数据第一阶段模型预测结果

In [ ]:


import os
import sys
import sys
sys.path.append('/tf/libo/Project/im/exe')
from my_sentence_transformers import SentenceTransformer, LoggingHandler
import numpy as np
import logging
import pandas as pd
from config_train import *

print('*'*200,'eva_s1')
print('***********currentD:',currentD)
path_list=path_list_1
model_dir=fst_model_dir
data_dir=fst_data_dir


import torch
from torch import Tensor
def pytorch_cos_sim(a: Tensor, b: Tensor):
    if not isinstance(a, torch.Tensor):
        a = torch.tensor(a)

    if not isinstance(b, torch.Tensor):
        b = torch.tensor(b)

    if len(a.shape) == 1:
        a = a.unsqueeze(0)

    if len(b.shape) == 1:
        b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1))

print('**load model from',model_dir)
##
def cos_sim(vector_a, vector_b):
    """
    计算两个向量之间的余弦相似度
    :param vector_a: 向量 a
    :param vector_b: 向量 b
    :return: sim
    """
    vector_a = np.mat(vector_a)
    vector_b = np.mat(vector_b)
    num = float(vector_a * vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num / denom
    sim = 0.5 + 0.5 * cos
    return sim

def cos_sim_1(vector_a, vector_b):
    """
    计算两个向量之间的余弦相似度
    :param vector_a: 向量 a
    :param vector_b: 向量 b
    :return: sim
    """
    vector_a = np.mat(vector_a)
    vector_b = np.mat(vector_b)
    num = float(vector_a * vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num / denom
    return cos

def cos_sim4matrix(arr, brr):
    return 0.5 + 0.5 * (arr.dot(brr.T) / (np.sqrt(np.sum(arr * arr)) * np.sqrt(np.sum(brr * brr, axis = 1))))


#### Just some code to print debug information to stdout
np.set_printoptions(threshold=100)

logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

# 加载模型
print('模型地址', model_dir)
model = SentenceTransformer(model_dir)


print('读取标准问及其向量'.center(60, '-'))
q_std_list=pd.read_csv(os.path.join(data_dir,fst_q_std_file),sep="\t",names=['c']).c.tolist()
q_std_sentence_embeddings=np.load(os.path.join(data_dir,fst_q_std_vectors_file))
print('标准问数量：——>',q_std_sentence_embeddings.shape,len(q_std_list))

print('读取所有语料及其向量'.center(60, '-'))
q_all=pd.read_csv(os.path.join(data_dir,fst_q_corpus_file),sep="\t",names=['c']).c.tolist()
q_all_sentence_embeddings=np.load(os.path.join(data_dir,fst_q_corpus_vectors_file))
q_all_sentence_embeddings_dict={q_all[i]:q_all_sentence_embeddings[i] for i in range(0,len(q_all))}
print('所有问句数量：——>',q_all_sentence_embeddings.shape,len(q_all))
# # 记得选择是v1还是v2
# # v1
q_corpus=q_std_list
corpus_sentence_embeddings=q_std_sentence_embeddings
dict_2={v:v for v in q_std_list}
pred2std_dict=dict_2
len(pred2std_dict)


for i in range(0,len(path_list)):
    print(f'开始评估新语料: {i}'.center(120, '='))
    df_k=pd.read_csv(path_list[i][0],sep="\t")
    df_k=df_k[~pd.isna(df_k.q_sim)]
    outputpath=path_list[i][1]
    print(path_list[i][0])
    print(outputpath)

    df_k['ifin']=df_k.q_std.apply(lambda v:1 if v in q_std_list  else 0)
    print("目标语料标问是否存在：——>",df_k.groupby("ifin")["ifin"].count())
    print("目标语料数量：",df_k.shape,'标问数量：',df_k.drop_duplicates("q_std").shape[0],'相似问数量：',df_k.drop_duplicates("q_sim").shape[0],'标语料去重后数量',df_k.drop_duplicates(["q_std","q_sim"]).shape[0])
    texts=df_k.q_sim.tolist()
    texts_in=[v for v in texts if v in q_all_sentence_embeddings_dict.keys()]
    texts_out=[v for v in texts if v not in q_all_sentence_embeddings_dict.keys()]
    texts_out_embeddings=model.encode(texts_out,batch_size=64)
    texts_embeddings_dict_1={texts_in[i]:q_all_sentence_embeddings_dict[texts_in[i]] for i in range(0,len(texts_in))}
    texts_embeddings_dict_2={texts_out[i]:texts_out_embeddings[i]  for i in range(0,len(texts_out))}
    texts_embeddings_dict={**texts_embeddings_dict_1, **texts_embeddings_dict_2}
    print('目标语料编码后数量：——>',len(texts_embeddings_dict))

    ## v1 对于都是有一个是小量的情况下
    K=10
    x_texts=texts
    x_texts_embeddings=np.array([texts_embeddings_dict[x_text] for x_text in x_texts])
    cos_scores = pytorch_cos_sim(x_texts_embeddings,corpus_sentence_embeddings).cpu()
    cos_scores_top_k_values, cos_scores_top_k_idx = torch.topk(cos_scores, K, dim=1, largest=True, sorted=False)
    cos_scores_top_k_values = cos_scores_top_k_values.tolist()
    cos_scores_top_k_idx = cos_scores_top_k_idx.tolist()
    cos_q_corpus_sort=[[q_corpus[v] for v in vlist] for vlist in cos_scores_top_k_idx ]
    result=[list(zip(cos_q_corpus_sort[i],cos_scores_top_k_values[i])) for i in range(0,len(x_texts))]
    texts_topk_dict={texts[i]:result[i] for i in range(0,len(texts))}
    #
    df_k['q_std_pred_list']=df_k.q_sim.map(texts_topk_dict)
    df_k.loc[:,'q_std_pred']=df_k.q_std_pred_list.apply(lambda v:v[0][0])
    df_k.loc[:,'prob']=df_k.q_std_pred_list.apply(lambda v:v[0][1])
    # df_k.loc[:,'q_std_pred_list_pair']=df_k.apply(lambda row: [(row['q_std'],row['q_sim'],v[0],v[1]) for v in row['q_std_pred_list']],axis=1)
    df_k['q_std_pred_list_v1']=df_k.q_std_pred_list.apply(lambda v:[k[0] for k in v])
    df_k['q_std_pred_list_v2']=df_k.q_std_pred_list.apply(lambda v:[k[1] for k in v])
    df_k['t1']=df_k.apply(lambda row:1 if row['q_std'] in row['q_std_pred_list_v1'][0:1] else 0,axis=1)
    df_k['t3']=df_k.apply(lambda row:1 if row['q_std'] in row['q_std_pred_list_v1'][0:3] else 0,axis=1)
    df_k['t5']=df_k.apply(lambda row:1 if row['q_std'] in row['q_std_pred_list_v1'][0:5] else 0,axis=1)
    df_k['t10']=df_k.apply(lambda row:1 if row['q_std'] in row['q_std_pred_list_v1'][0:10] else 0,axis=1)

    print('目标语料准确率：——>')
    print(df_k.shape)
    print(df_k.t1.sum()/df_k.shape[0],df_k.t3.sum()/df_k.shape[0],df_k.t5.sum()/df_k.shape[0],df_k.t10.sum()/df_k.shape[0])
    print('目标语料准确率[有效标问]：——>')
    df_k_need=df_k[df_k.ifin==1]
    print(df_k_need.shape)
    print(df_k_need.t1.sum()/df_k_need.shape[0],df_k_need.t3.sum()/df_k_need.shape[0],df_k_need.t5.sum()/df_k_need.shape[0],df_k_need.t10.sum()/df_k_need.shape[0])

    print(outputpath)

    df_k.to_csv(outputpath,index=None,sep="\t")

### 获得所有待测数据第二阶段模型预测结果

In [ ]:
import os

from task_sentence_embedding_FinanceFAQ_step2_1 import model
import numpy as np
import logging
import pandas as pd
from config import *

path_list=path_list_2
model_dir=sec_model_dir
data_dir=sec_data_dir


import torch
from torch import Tensor
def pytorch_cos_sim(a: Tensor, b: Tensor):
    if not isinstance(a, torch.Tensor):
        a = torch.tensor(a)

    if not isinstance(b, torch.Tensor):
        b = torch.tensor(b)

    if len(a.shape) == 1:
        a = a.unsqueeze(0)

    if len(b.shape) == 1:
        b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1))
print('**load model from',model_dir)
##
def cos_sim(vector_a, vector_b):
    """
    计算两个向量之间的余弦相似度
    :param vector_a: 向量 a
    :param vector_b: 向量 b
    :return: sim
    """
    vector_a = np.mat(vector_a)
    vector_b = np.mat(vector_b)
    num = float(vector_a * vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num / denom
    sim = 0.5 + 0.5 * cos
    return sim

def cos_sim_1(vector_a, vector_b):
    """
    计算两个向量之间的余弦相似度
    :param vector_a: 向量 a
    :param vector_b: 向量 b
    :return: sim
    """
    vector_a = np.mat(vector_a)
    vector_b = np.mat(vector_b)
    num = float(vector_a * vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num / denom
    return cos

def cos_sim4matrix(arr, brr):
    return 0.5 + 0.5 * (arr.dot(brr.T) / (np.sqrt(np.sum(arr * arr)) * np.sqrt(np.sum(brr * brr, axis = 1))))


#### Just some code to print debug information to stdout
np.set_printoptions(threshold=100)

logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

# 加载模型
model = SentenceTransformer(model_dir)

q_std_list=pd.read_csv(os.path.join(data_dir,sec_q_std_file),sep="\t",names=['c']).c.tolist()
q_std_sentence_embeddings=np.load(os.path.join(data_dir,sec_q_std_vectors_file))
print('标问数量：——>',q_std_sentence_embeddings.shape,len(q_std_list))
q_all=pd.read_csv(os.path.join(data_dir,sec_q_corpus_file),sep="\t",names=['c']).c.tolist()
q_all_sentence_embeddings=np.load(os.path.join(data_dir,sec_q_corpus_vectors_file))
q_all_sentence_embeddings_dict={q_all[i]:q_all_sentence_embeddings[i] for i in range(0,len(q_all))}
print('所有问句数量：——>',q_all_sentence_embeddings.shape,len(q_all))
# # 记得选择是v1还是v2
# # v1
q_corpus=q_std_list
corpus_sentence_embeddings=q_std_sentence_embeddings
corpus_sentence_embeddings_dict={q_corpus[i]:corpus_sentence_embeddings[i] for i in range(0,len(q_corpus))}
dict_2={v:v for v in q_std_list}
pred2std_dict=dict_2


for i in range(0,len(path_list)):  
    print(f'开始评估新语料: {i}'.center(120, '='))
    # # # ##v1
    df_k=pd.read_csv(path_list[i][0],sep="\t")
    outputpath=path_list[i][1]
    print(path_list[i][0])
    print(outputpath)
    
    texts=df_k.q_sim.tolist()
    texts_in=[v for v in texts if v in q_all_sentence_embeddings_dict.keys()]
    texts_out=[v for v in texts if v not in q_all_sentence_embeddings_dict.keys()]
    texts_out_embeddings=model.encode(texts_out,batch_size=64)
    texts_embeddings_dict_1={texts_in[i]:q_all_sentence_embeddings_dict[texts_in[i]] for i in range(0,len(texts_in))}
    texts_embeddings_dict_2={texts_out[i]:texts_out_embeddings[i]  for i in range(0,len(texts_out))}
    texts_embeddings_dict={**texts_embeddings_dict_1, **texts_embeddings_dict_2}
    print('目标语料编码数量：——>',len(texts_embeddings_dict))

    def cos_sim4matrix_2(arr, brr):
        return (arr.dot(brr.T) / (np.sqrt(np.sum(arr * arr)) * np.sqrt(np.sum(brr * brr, axis = 1))))

    def get_sec_result(text,std_texts):
        a_text_embeddings=texts_embeddings_dict[text]
        b_text_embeddings=np.array([corpus_sentence_embeddings_dict[v] for v in std_texts])
    #     print(a_text_embeddings.shape,b_text_embeddings.shape)
        sims_with_std=cos_sim4matrix_2(a_text_embeddings,b_text_embeddings).reshape(-1)
        sort_idx=np.argsort(-sims_with_std).tolist()
        intents_sort=[std_texts[idx] for idx in sort_idx]
        sims_values=[sims_with_std[idx] for idx in sort_idx]
        result=list(zip(intents_sort,sims_values))
        return(result)
    # get_sec_result(text,std_texts)

    df_k['q_std_pred_list_v1']=df_k.q_std_pred_list_v1.apply(lambda v:eval(v))

    df_k['q_std_pred_list_2']=df_k.apply(lambda row:get_sec_result(row['q_sim'],row['q_std_pred_list_v1']),axis=1)


    df_k['q_std_pred_list_2_v1']=df_k.q_std_pred_list_2.apply(lambda v:[k[0] for k in v])
    df_k['q_std_pred_list_2_v2']=df_k.q_std_pred_list_2.apply(lambda v:[k[1] for k in v])
    df_k['q_std_pred_2']=df_k.q_std_pred_list_2_v1.apply(lambda v:v[0])
    df_k['prob_2']=df_k.q_std_pred_list_2_v2.apply(lambda v:v[0])


    df_k['r1']=df_k.apply(lambda row:1 if row['q_std'] in row['q_std_pred_list_2_v1'][0:1] else 0,axis=1)
    df_k['r3']=df_k.apply(lambda row:1 if row['q_std'] in row['q_std_pred_list_2_v1'][0:3] else 0,axis=1)
    df_k['r5']=df_k.apply(lambda row:1 if row['q_std'] in row['q_std_pred_list_2_v1'][0:5] else 0,axis=1)
    df_k['r10']=df_k.apply(lambda row:1 if row['q_std'] in row['q_std_pred_list_2_v1'][0:10] else 0,axis=1)


    ##扣除不包含的标准问
    print('目标语料准确率：——>')
    print(df_k.shape)
    df_1=df_k
    print('第一阶段整体准确率',df_1.t1.sum()/df_1.shape[0],df_1.t3.sum()/df_1.shape[0],df_1.t5.sum()/df_1.shape[0],df_1.t10.sum()/df_1.shape[0])
    df_2=df_k[df_k.t10==1]
    print('第二阶段整体准确率',df_2.r1.sum()/df_2.shape[0],df_2.r3.sum()/df_2.shape[0],df_2.r5.sum()/df_2.shape[0],df_2.r10.sum()/df_2.shape[0])
    df_3=df_k
    print('整体准确率',df_3.r1.sum()/df_3.shape[0],df_3.r3.sum()/df_3.shape[0],df_3.r5.sum()/df_3.shape[0],df_3.r10.sum()/df_3.shape[0])


    ##扣除不包含的标准问
    print('目标语料准确率[有效标问]：——>')
    df_k_need=df_k[df_k.ifin==1]
    print(df_k_need.shape)
    df_1=df_k_need
    print('第一阶段整体准确率',df_1.t1.sum()/df_1.shape[0],df_1.t3.sum()/df_1.shape[0],df_1.t5.sum()/df_1.shape[0],df_1.t10.sum()/df_1.shape[0])
    df_2=df_k_need[df_k_need.t10==1]
    print('第二阶段整体准确率',df_2.r1.sum()/df_2.shape[0],df_2.r3.sum()/df_2.shape[0],df_2.r5.sum()/df_2.shape[0],df_2.r10.sum()/df_2.shape[0])
    df_3=df_k_need
    print('整体准确率',df_3.r1.sum()/df_3.shape[0],df_3.r3.sum()/df_3.shape[0],df_3.r5.sum()/df_3.shape[0],df_3.r10.sum()/df_3.shape[0])

    print(outputpath)
    df_k.to_csv(outputpath,index=None,sep="\t")
	


### 获得单例文本预测结果

In [ ]:
import pandas as pd
import os
import sys
sys.path.append('/tf/libo/Project/im/exe')
from my_sentence_transformers import SentenceTransformer, LoggingHandler
# from my_sentence_transformers import SentenceTransformer, LoggingHandler
import numpy as np
import logging
import torch.nn.functional as F
import datetime
import sys
import torch

import config_train as config

print('***********currentD:',config.currentD)

# currentD='20210805'
text="我想开户"
# # v1
step1_model_dir="/tf/yy/im_all_0823/tm/output/%s_fst_d1/m2/"%config.currentD
step1_data_dir="/tf/yy/im_all_0823/tm/data_train/%s_fst_d1"%config.currentD
step2_model_dir="/tf/yy/im_all_0823/tm/output/%s_sec_d1/m2/"%config.currentD
step2_data_dir="/tf/yy/im_all_0823/tm/data_train/%s_sec_d1"%config.currentD
# # # v2
# step1_model_dir="/tf/yy/prd/%s/topk_model_info_1/model_dir"%config.currentD
# step1_data_dir="/tf/yy/prd/%s/topk_model_info_1"%config.currentD
# step2_model_dir="/tf/yy/prd/%s/topk_model_info_2/model_dir"%config.currentD
# step2_data_dir="/tf/yy/prd/%s/topk_model_info_2"%config.currentD

def set_model_gpu(setgpu):
    if setgpu==True:
        print('open GPU')
        os.system('nvidia-smi -q -d Memory |grep -A4 GPU|grep Free >tmp')
        memory_gpu=[int(x.split()[2]) for x in open('tmp','r').readlines()]
        gpu_n=str(np.argmax(memory_gpu))
        os.environ['CUDA_VISIBLE_DEVICES']='0'
        print("use gpu %s"%gpu_n)
#         memory_gpu=[]
    else:
        print('shut down GPU')
        os.environ['CUDA_VISIBLE_DEVICES']='-1'
    os.system('rm tmp')


    
class get_sim_first():
    def __init__(self,step1_model_dir,step1_data_dir,step1_q_std_vector_filename):
        self.model_dir=step1_model_dir
        self.data_dir=step1_data_dir
        self.q_std_filename=config.fst_q_std_file
        self.q_std_vector_filename=step1_q_std_vector_filename
        self.q_std_path=os.path.join(self.data_dir,self.q_std_filename)
        ## set gpu
        
        ## load std questions
        with open(self.q_std_path,'r',encoding="utf-8") as f:
            data=f.readlines()
        self.q_std_list=[v.strip() for v in data]
        ## load std questions' vectors
        self.std_sentence_embeddings=np.load(os.path.join(self.data_dir,self.q_std_vector_filename))
        ## load model
        device = "cuda:0" if torch.cuda.is_available() else "cpu"
        self.model = SentenceTransformer(self.model_dir,device=device)
        ## 
        
    def cos_sim(self,vector_a,vector_b):
        vector_a = np.mat(vector_a)
        vector_b = np.mat(vector_b)
        num = float(vector_a * vector_b.T)
        denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
        cos = num / denom
        sim = 0.5 + 0.5 * cos
        return sim
    
    def cos_sim4matrix(self,arr, brr):
        return 0.5 + 0.5 * (arr.dot(brr.T) / (np.sqrt(np.sum(arr * arr)) * np.sqrt(np.sum(brr * brr, axis = 1))))
    def cos_sim4matrix_2(self,arr, brr):
        return (arr.dot(brr.T) / (np.sqrt(np.sum(arr * arr)) * np.sqrt(np.sum(brr * brr, axis = 1))))
    
    def get_topK(self,text,K=20):
        text_embedding=self.model.encode([text])[0]
#         sims_with_std=np.array([self.cos_sim(vec,text_embedding) for vec in self.std_sentence_embeddings])
        sims_with_std=np.array(self.cos_sim4matrix_2(text_embedding,self.std_sentence_embeddings))
        sort_idx=np.argsort(-sims_with_std)[:K]
        sims_q_sort=[self.q_std_list[idx] for idx in sort_idx]
        sims_values=[sims_with_std[idx] for idx in sort_idx]
        result=list(zip(sims_q_sort,sims_values))
        return(result)
  
    
class get_sim_second():
    def __init__(self,step2_model_dir,step2_data_dir,step2_q_std_vector_filename):
        self.model_dir=step2_model_dir
        self.data_dir=step2_data_dir
        self.q_std_filename=config.sec_q_std_file
        self.q_std_vector_filename=step2_q_std_vector_filename
        self.q_std_path=os.path.join(self.data_dir,self.q_std_filename)
        
        ## set gpu
        
        ## load std questions
        with open(self.q_std_path,'r',encoding="utf-8") as f:
            data=f.readlines()
        self.q_std_list=[v.strip() for v in data]
        ## load std questions' vectors
        self.std_sentence_embeddings=np.load(os.path.join(self.data_dir,self.q_std_vector_filename))
        self.corpus_sentence_embeddings_dict_sec=dict(list(zip(self.q_std_list,self.std_sentence_embeddings)))
        ## load model
        device = "cuda:0" if torch.cuda.is_available() else "cpu"
        self.model = SentenceTransformer(self.model_dir,device=device)
        ## 
        
    def cos_sim(self,vector_a,vector_b):
        vector_a = np.mat(vector_a)
        vector_b = np.mat(vector_b)
        num = float(vector_a * vector_b.T)
        denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
        cos = num / denom
        sim = 0.5 + 0.5 * cos
        return sim
    
    def cos_sim4matrix(self,arr, brr):
        return 0.5 + 0.5 * (arr.dot(brr.T) / (np.sqrt(np.sum(arr * arr)) * np.sqrt(np.sum(brr * brr, axis = 1))))
    def cos_sim4matrix_2(self,arr, brr):
        return (arr.dot(brr.T) / (np.sqrt(np.sum(arr * arr)) * np.sqrt(np.sum(brr * brr, axis = 1))))
    
    def get_topK(self,text,K=20):
        text_embedding=self.model.encode([text])[0]
#         sims_with_std=np.array([self.cos_sim(vec,text_embedding) for vec in self.std_sentence_embeddings])
        sims_with_std=np.array(self.cos_sim4matrix_2(text_embedding,self.std_sentence_embeddings))
        sort_idx=np.argsort(-sims_with_std)[:K]
        sims_q_sort=[self.q_std_list[idx] for idx in sort_idx]
        sims_values=[sims_with_std[idx] for idx in sort_idx]
        result=list(zip(sims_q_sort,sims_values))
        return(result)
 

set_model_gpu(setgpu=True) 


import torch
from torch import Tensor
def pytorch_cos_sim(a: Tensor, b: Tensor):
    if not isinstance(a, torch.Tensor):
        a = torch.tensor(a)

    if not isinstance(b, torch.Tensor):
        b = torch.tensor(b)

    if len(a.shape) == 1:
        a = a.unsqueeze(0)

    if len(b.shape) == 1:
        b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1))



get_sim_step1=get_sim_first(config.fst_model_dir,config.fst_data_dir,config.fst_q_std_vectors_file)
get_sim_step2=get_sim_second(config.sec_model_dir,config.sec_data_dir,config.sec_q_std_vectors_file)

print('标准问数量',len(get_sim_step1.q_std_list),len(get_sim_step2.q_std_list))

def get_sec_result(text,std_texts):
    a_text_embeddings=x_texts_embeddings_dict_2[text]
    b_text_embeddings=np.array([get_sim_step2.corpus_sentence_embeddings_dict_sec[v] for v in std_texts])
#     print(a_text_embeddings.shape,b_text_embeddings.shape)
    sims_with_std=cos_sim4matrix_2(a_text_embeddings,b_text_embeddings).reshape(-1)
    sort_idx=np.argsort(-sims_with_std).tolist()
    intents_sort=[std_texts[idx] for idx in sort_idx]
    sims_values=[sims_with_std[idx] for idx in sort_idx]
    result=list(zip(intents_sort,sims_values))
    return(result)
# get_sec_result(text,std_texts)



# 第一阶段
time1=datetime.datetime.now();print(time1)
print('************',text)
result_first=get_sim_step1.get_topK(text=text,K=10)
print('第一阶段\n',result_first[0:20])
first_intents=[v[0] for v in result_first]
a_texts_embeddings_2=np.array(get_sim_step2.model.encode([text]))
b_texts_embeddings_2=np.array([get_sim_step2.corpus_sentence_embeddings_dict_sec[v] for v in first_intents])
sims_with_std=get_sim_step2.cos_sim4matrix_2(a_texts_embeddings_2,b_texts_embeddings_2).reshape(-1)
sort_idx=np.argsort(-sims_with_std).tolist()
intents_sort=[first_intents[idx] for idx in sort_idx]
sims_values=[sims_with_std[idx] for idx in sort_idx]
result_second=list(zip(intents_sort,sims_values))

time2=datetime.datetime.now();print(time2)
time_diff=time2-time1;print('cost time:',time_diff.seconds)

print('*************************')
print('第二阶段\n',result_second[0:20])

# # x='电脑版官方网站'
# y='钠离子电池对行业的影响'
# print(get_sim_step2.cos_sim4matrix_2(np.array(get_sim_step1.model.encode([text])),np.array(get_sim_step1.model.encode([y]))))
# # print('x',np.array(get_sim_step2.model.encode([x]))
# # print('y',np.array(get_sim_step2.model.encode([y]))
# # print(get_sim_step2.cos_sim4matrix_2(np.array(get_sim_step2.model.encode([x])),np.array(get_sim_step2.model.encode([y]))))